In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid

from network import FeedForwardDenseNetwork
import activation
import metrics
import losses
import normalizers

In [9]:
def generate_data(size: int):
    data = np.random.uniform(low=0, high=1, size=(2, size))
    data = data * np.random.choice([-1, 1], size=(2, size))
    labels = (np.sign(data[0]) == np.sign(data[1])).astype(int)
    return data, labels

In [10]:
train_data, train_labels = generate_data(50_000)
train_data.shape, train_labels.shape

((2, 50000), (50000,))

In [11]:
test_data, test_labels = generate_data(10_000)
test_data.shape, test_labels.shape

((2, 10000), (10000,))

In [12]:
NEURONS = [2, 4, 1]
METRIC = metrics.Accuracy
LOSS = losses.MSE

params_dict = {
    'activation': [activation.Relu, activation.Sigmoid],
    'normalizer': [None, normalizers.L1, normalizers.L2],
    'learning_rate': [0.001, 0.005, 0.01]
}
params_dict

{'activation': [activation.Relu, activation.Sigmoid],
 'normalizer': [None, normalizers.L1, normalizers.L2],
 'learning_rate': [0.001, 0.005, 0.01]}

In [13]:
param_grid = pd.DataFrame(ParameterGrid(params_dict))
param_grid

,activation,learning_rate,normalizer
0,<class 'activation.Relu'>,0.001,None
1,<class 'activation.Relu'>,0.001,<class 'normalizers.L1'>
2,<class 'activation.Relu'>,0.001,<class 'normalizers.L2'>
3,<class 'activation.Relu'>,0.005,None
4,<class 'activation.Relu'>,0.005,<class 'normalizers.L1'>
5,<class 'activation.Relu'>,0.005,<class 'normalizers.L2'>
6,<class 'activation.Relu'>,0.010,None
7,<class 'activation.Relu'>,0.010,<class 'normalizers.L1'>
8,<class 'activation.Relu'>,0.010,<class 'normalizers.L2'>
9,<class 'activation.Sigmoid'>,0.001,None


In [14]:
model_dict = {
'model': [],
'loss': [],
'accuracy': []
}

for i in param_grid.index:
# instantiate the model
    model = FeedForwardDenseNetwork(
        neurons=NEURONS,
        activation=param_grid.loc[i, 'activation'],
        metric=METRIC,
        loss=LOSS,
        normalizer=param_grid.loc[i, 'normalizer']
    )

    # fit the model
    model.fit(
        X=train_data, 
        Y=train_labels, 
        learning_rate=param_grid.loc[i, 'learning_rate'],
        verbose=False
    )

    # evaluate the model
    loss, accuracy = model.evaluate(test_data, test_labels)
    model_dict['model'].append(model)
    model_dict['loss'].append(loss)
    model_dict['accuracy'].append(accuracy)

    print(f"{i}: {loss = }, {accuracy = }") 

0: loss = 0.3681, accuracy = 0.6319
1: loss = 0.4502, accuracy = 0.5498
2: loss = 0.5277, accuracy = 0.4723
3: loss = 0.3336, accuracy = 0.6664
4: loss = 0.3828, accuracy = 0.6172
5: loss = 0.5398, accuracy = 0.4602
6: loss = 0.185, accuracy = 0.815
7: loss = 0.2174, accuracy = 0.7826
8: loss = 0.3171, accuracy = 0.6829
9: loss = 0.4984, accuracy = 0.5016
10: loss = 0.5049, accuracy = 0.4951
11: loss = 0.4984, accuracy = 0.5016
12: loss = 0.5556, accuracy = 0.4444
13: loss = 0.4986, accuracy = 0.5014
14: loss = 0.4984, accuracy = 0.5016
15: loss = 0.4937, accuracy = 0.5063
16: loss = 0.4893, accuracy = 0.5107
17: loss = 0.6013, accuracy = 0.3987


In [15]:
model_grid = pd.DataFrame(model_dict)
model_grid

,model,loss,accuracy
0,<network.FeedForwardDenseNetwork object at 0x0...,0.3681,0.6319
1,<network.FeedForwardDenseNetwork object at 0x0...,0.4502,0.5498
2,<network.FeedForwardDenseNetwork object at 0x0...,0.5277,0.4723
3,<network.FeedForwardDenseNetwork object at 0x0...,0.3336,0.6664
4,<network.FeedForwardDenseNetwork object at 0x0...,0.3828,0.6172
5,<network.FeedForwardDenseNetwork object at 0x0...,0.5398,0.4602
6,<network.FeedForwardDenseNetwork object at 0x0...,0.1850,0.8150
7,<network.FeedForwardDenseNetwork object at 0x0...,0.2174,0.7826
8,<network.FeedForwardDenseNetwork object at 0x0...,0.3171,0.6829
9,<network.FeedForwardDenseNetwork object at 0x0...,0.4984,0.5016


In [16]:
best_model = model_grid.loc[model_grid['accuracy'].idxmax()]
best_model.model.summary()
best_model.loss, best_model.accuracy

Number of layers: 3
Number of neurons per layer:
	1: 2
	2: 4
	3: 1
Activation: relu
Metric: accuracy
Loss: mean_sqared_error
Normalization: none


(0.185, 0.815)